<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/pix2pix/controlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/lllyasviel/ControlNet.git

Cloning into 'ControlNet'...
remote: Enumerating objects: 820, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 820 (delta 105), reused 87 (delta 83), pack-reused 665
Receiving objects: 100% (820/820), 84.33 MiB | 33.10 MiB/s, done.
Resolving deltas: 100% (250/250), done.


In [ ]:
# requirements.txt
"""
gradio
albumentations
opencv-contrib-python
imageio
imageio-ffmpeg
pytorch-lightning
omegaconf
test-tube
streamlit
einops
transformers
webdataset
kornia
open_clip_torch
invisible-watermark
streamlit-drawable-canvas
torchmetrics
timm
addict
yapf
prettytable
"""

In [ ]:
%cd ControlNet
!pip install -r requirements.txt

In [ ]:
!pip install xformers triton

In [ ]:
#@title canny
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth -P models
# !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -P models
# !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth -P models
# !wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth -P models

In [ ]:
!python gradio_canny2image.py

In [ ]:
#@title pose
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth -P models

In [ ]:
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/body_pose_model.pth -P annotator/ckpts

In [ ]:
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/hand_pose_model.pth -P annotator/ckpts

In [ ]:
# Please change gradio_pose2image.py -> block.launch(share=True)
!python gradio_pose2image.py

In [ ]:
#@title mlsd
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth -P models

In [ ]:
!wget https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_large_512_fp32.pth -P annotator/ckpts

In [29]:
!python gradio_hough2image.py

2023-02-15 01:22:48.337526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 01:22:49.379907: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-15 01:22:49.380027: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-15 01:22:49.380056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War

In [ ]:
#@title Advanced 
!wget https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-full.safetensors -P models

In [ ]:
!wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt -P models

In [ ]:
!nvidia-smi

In [25]:
# cannot transfer because of out of memory
!python tool_transfer_control.py

2023-02-15 01:13:06.248756: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 01:13:10.105106: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-15 01:13:10.105784: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-15 01:13:10.105809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War